In [524]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [525]:
pd.set_option('display.max_columns', None)
current_dir = os.getcwd()

## DATA

In [526]:
url_sales = os.path.join(current_dir, "../data/raw/processed_data.csv")
sales = pd.read_csv(url_sales, sep=",")

C:\Users\andta\AppData\Local\Temp\ipykernel_5196\792020892.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv(url_sales, sep=",")


In [527]:
sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,SalesDate,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22 10:29:00.960,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11 17:52:45.710,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19 23:14:01.500,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06 00:16:27.280,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30 21:06:49.980,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR


In [528]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6758125 entries, 0 to 6758124
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CategoryID         int64  
 1   CategoryName       object 
 2   ProductName        object 
 3   Price              float64
 4   Class              object 
 5   ModifyDate         object 
 6   Resistant          object 
 7   IsAllergic         object 
 8   VitalityDays       float64
 9   SalesID            int64  
 10  CustomerID         int64  
 11  Quantity           int64  
 12  ProductID          int64  
 13  Discount           float64
 14  TotalPrice         float64
 15  SalesDate          object 
 16  TransactionNumber  object 
 17  FirstName          object 
 18  MiddleInitial      object 
 19  LastName           object 
 20  CityID             int64  
 21  Address            object 
 22  CityName           object 
 23  Zipcode            int64  
 24  CountryID          int64  
 25  CountryName       

In [529]:
sales.shape

(6758125, 27)

#### Modificar la columna fecha para quedarnos con solo la fecha y descartar la hora en la columa que nos interesa:

#### Sales date

In [530]:
sales['SalesDate'] = pd.to_datetime(sales['SalesDate']).dt.date

sales.rename(columns = {"SalesDate" : "Date" }, inplace = True)

sales['Date']

0          2018-02-22
1          2018-01-11
2          2018-04-19
3          2018-04-06
4          2018-04-30
              ...    
6758120    2018-03-06
6758121    2018-03-02
6758122    2018-03-04
6758123    2018-03-12
6758124    2018-01-13
Name: Date, Length: 6758125, dtype: object

#### Crear columna de estados segun ciudades

In [531]:
sales['CityName'].unique()

array(['Little Rock', 'Toledo', 'Oakland', 'Akron', 'Indianapolis',
       'Stockton', 'Baltimore', 'Dallas', 'Albuquerque', 'Anaheim',
       'Colorado', 'Honolulu', 'Buffalo', 'Lincoln', 'Omaha', 'Mobile',
       'Phoenix', 'Newark', 'Houston', 'San Francisco', 'San Jose',
       'Charlotte', 'Glendale', 'Grand Rapids', 'Denver', 'New York',
       'Cincinnati', 'Tulsa', 'Cleveland', 'Yonkers', 'Montgomery',
       'St. Paul', 'Riverside', 'Austin', 'St. Petersburg', 'Tacoma',
       'Seattle', 'Lubbock', 'Louisville', 'Spokane', 'Santa Ana',
       'Long Beach', 'Garland', 'Bakersfield', 'Oklahoma', 'Dayton',
       'Milwaukee', 'Hialeah', 'Fresno', 'Richmond', 'Shreveport',
       'Baton Rouge', 'Minneapolis', 'Fort Wayne', 'Des Moines',
       'Fort Worth', 'Kansas', 'Mesa', 'San Antonio', 'Nashville',
       'Portland', 'Atlanta', 'Aurora', 'Fremont', 'Washington',
       'Philadelphia', 'Greensboro', 'St. Louis', 'San Diego', 'Jackson',
       'Tucson', 'Pittsburgh', 'Boston', '

In [532]:
# Creamos un diccionario que mapea cada ciudad a su estado correspondiente

city_to_state = {
    'Little Rock': 'Arkansas',
    'Toledo': 'Ohio',
    'Oakland': 'California',
    'Akron': 'Ohio',
    'Indianapolis': 'Indiana',
    'Stockton': 'California',
    'Baltimore': 'Maryland',
    'Dallas': 'Texas',
    'Albuquerque': 'New Mexico',
    'Anaheim': 'California',
    'Honolulu': 'Hawaii',
    'Buffalo': 'New York',
    'Lincoln': 'Nebraska',
    'Omaha': 'Nebraska',
    'Mobile': 'Alabama',
    'Phoenix': 'Arizona',
    'Newark': 'New Jersey',
    'Houston': 'Texas',
    'San Francisco': 'California',
    'San Jose': 'California',
    'Charlotte': 'North Carolina',
    'Glendale': 'Arizona',
    'Grand Rapids': 'Michigan',
    'Denver': 'Colorado',
    'New York': 'New York',
    'Cincinnati': 'Ohio',
    'Tulsa': 'Oklahoma',
    'Cleveland': 'Ohio',
    'Yonkers': 'New York',
    'Montgomery': 'Alabama',
    'St. Paul': 'Minnesota',
    'Riverside': 'California',
    'Austin': 'Texas',
    'St. Petersburg': 'Florida',
    'Tacoma': 'Washington',
    'Seattle': 'Washington',
    'Lubbock': 'Texas',
    'Louisville': 'Kentucky',
    'Spokane': 'Washington',
    'Santa Ana': 'California',
    'Long Beach': 'California',
    'Garland': 'Texas',
    'Bakersfield': 'California',
    'Oklahoma': 'Oklahoma',
    'Dayton': 'Ohio',
    'Milwaukee': 'Wisconsin',
    'Hialeah': 'Florida',
    'Fresno': 'California',
    'Richmond': 'Virginia',
    'Shreveport': 'Louisiana',
    'Baton Rouge': 'Louisiana',
    'Minneapolis': 'Minnesota',
    'Fort Wayne': 'Indiana',
    'Des Moines': 'Iowa',
    'Fort Worth': 'Texas',
    'Kansas': 'Kansas',
    'Mesa': 'Arizona',
    'San Antonio': 'Texas',
    'Nashville': 'Tennessee',
    'Portland': 'Oregon',
    'Atlanta': 'Georgia',
    'Aurora': 'Colorado',
    'Colorado': 'Colorado',
    'Fremont': 'California',
    'Washington': 'District of Columbia',
    'Philadelphia': 'Pennsylvania',
    'Greensboro': 'North Carolina',
    'St. Louis': 'Missouri',
    'San Diego': 'California',
    'Jackson': 'Mississippi',
    'Tucson': 'Arizona',
    'Pittsburgh': 'Pennsylvania',
    'Boston': 'Massachusetts',
    'Columbus': 'Ohio',
    'Las Vegas': 'Nevada',
    'Rochester': 'New York',
    'El Paso': 'Texas',
    'Birmingham': 'Alabama',
    'Virginia Beach': 'Virginia',
    'Corpus Christi': 'Texas',
    'Los Angeles': 'California',
    'Memphis': 'Tennessee',
    'Raleigh': 'North Carolina',
    'Detroit': 'Michigan',
    'Wichita': 'Kansas',
    'Jacksonville': 'Florida',
    'Tampa': 'Florida',
    'Miami': 'Florida',
    'Jersey': 'New Jersey',
    'Arlington': 'Texas',
    'Chicago': 'Illinois',
    'New Orleans': 'Louisiana',
    'Sacramento': 'California',
    'Norfolk': 'Virginia',
    'Madison': 'Wisconsin',
    'Anchorage': 'Alaska'
}

# Aplicamos este diccionario para crear una nueva columna 'State' 

sales['State'] = sales['CityName'].map(city_to_state)

sales['State'].unique()


array(['Arkansas', 'Ohio', 'California', 'Indiana', 'Maryland', 'Texas',
       'New Mexico', 'Colorado', 'Hawaii', 'New York', 'Nebraska',
       'Alabama', 'Arizona', 'New Jersey', 'North Carolina', 'Michigan',
       'Oklahoma', 'Minnesota', 'Florida', 'Washington', 'Kentucky',
       'Wisconsin', 'Virginia', 'Louisiana', 'Iowa', 'Kansas',
       'Tennessee', 'Oregon', 'Georgia', 'District of Columbia',
       'Pennsylvania', 'Missouri', 'Mississippi', 'Massachusetts',
       'Nevada', 'Illinois', 'Alaska'], dtype=object)

In [533]:
sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR,Arkansas
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR,Ohio
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR,California
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR,Ohio
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR,Indiana


# Pib por estado + sales [OK]

In [534]:
url_pib = os.path.join(current_dir, "../data/raw/macrodata/pib_por_estados.csv")
pib = pd.read_csv(url_pib, sep=",")

pib.head()

,cp,estado,january,february,march,april,may
0,0,United States,6.681359e+15,6.681359e+15,6.681359e+15,6.716825e+15,6.716825e+15
1,1000,Alabama,7.335403e+13,7.335403e+13,7.335403e+13,7.339050e+13,7.339050e+13
2,2000,Alaska,1.763033e+13,1.763033e+13,1.763033e+13,1.748017e+13,1.748017e+13
3,4000,Arizona,1.145728e+14,1.145728e+14,1.145728e+14,1.149027e+14,1.149027e+14
4,5000,Arkansas,4.205223e+13,4.205223e+13,4.205223e+13,4.202373e+13,4.202373e+13


In [535]:
pib.rename(columns = {"estado" : "State" }, inplace = True)

In [536]:
# # Paso 1: Asegurarse de que la fecha esté en formato datetime
# sales['Date'] = pd.to_datetime(sales['Date'])

# # Paso 2: Extraer el nombre del mes en minúscula
# sales['mes_nombre'] = sales['Date'].dt.month_name().str.lower()

# # Paso 3: Crear una función que devuelva el valor correcto del PIB
# def obtener_pib(row):
#     estado = row['State']
#     mes = row['mes_nombre']
    
#     # Filtrar el DataFrame pib para ese estado
#     fila_pib = pib[pib['State'] == estado]
    
#     # Si encontramos el estado y el mes, devolvemos el valor
#     if not fila_pib.empty and mes in fila_pib.columns:
#         return fila_pib.iloc[0][mes]
#     else:
#         return None  

# # Paso 4: Aplicar la función a cada fila del DataFrame
# sales['pib'] = sales.apply(obtener_pib, axis=1)

# # (Opcional) Eliminar columna auxiliar
# sales.drop(columns='mes_nombre', inplace=True)

# sales.head()

# Riqueza per capita + sales [OK]

In [537]:
url_rpc = os.path.join(current_dir, "../data/raw/macrodata/riqueza_per_capita.csv")
rpc = pd.read_csv(url_rpc, sep=",")

rpc.head()

,cp,estado,january,february,march,april,may
0,0,UnitedStates,1.748100e+10,1.748100e+10,1.748100e+10,1.766200e+10,1.766200e+10
1,1000,Alabama,1.360833e+10,1.360833e+10,1.360833e+10,1.369800e+10,1.369800e+10
2,2000,Alaska,1.935867e+10,1.935867e+10,1.935867e+10,1.949667e+10,1.949667e+10
3,4000,Arizona,1.487600e+10,1.487600e+10,1.487600e+10,1.498700e+10,1.498700e+10
4,5000,Arkansas,1.414467e+10,1.414467e+10,1.414467e+10,1.425400e+10,1.425400e+10


In [538]:
rpc.rename(columns = {"estado" : "State" }, inplace = True)

In [539]:
# # Paso 1: Asegurarse de que la fecha esté en formato datetime
# sales['Date'] = pd.to_datetime(sales['Date'])

# # Paso 2: Extraer el nombre del mes en minúscula
# sales['mes_nombre'] = sales['Date'].dt.month_name().str.lower()

# # Paso 3: Crear una función que devuelva el valor correcto del PIB
# def obtener_rpc(row):
#     estado = row['State']
#     mes = row['mes_nombre']
    
#     # Filtrar el DataFrame pib para ese estado
#     fila_rpc = rpc[pib['State'] == estado]
    
#     # Si encontramos el estado y el mes, devolvemos el valor
#     if not fila_rpc.empty and mes in fila_rpc.columns:
#         return fila_rpc.iloc[0][mes]
#     else:
#         return None  

# # Paso 4: Aplicar la función a cada fila del DataFrame
# sales['rpc'] = sales.apply(obtener_rpc, axis=1)

# # (Opcional) Eliminar columna auxiliar
# sales.drop(columns='mes_nombre', inplace=True)

# sales.head()

# Wti Crude Oil + (((((SALES))))) [OK]

In [540]:
url_wti_crude_oil = os.path.join(current_dir, "../data/raw/macrodata/wti_crude_oil.csv")
wti_oil_raw = pd.read_csv(url_wti_crude_oil, sep = ",")
wti_oil_raw

,Date,Open,High,Low,Close,Adj Close,Volume
0,"May 8, 2018",7.02,7.24,6.59,7.22,7.0267,"5,904,900"
1,"May 7, 2018",6.80,7.35,6.79,6.96,6.7737,"5,220,500"
2,"May 4, 2018",6.49,6.80,6.36,6.64,6.4622,"3,152,800"
3,"May 3, 2018",6.60,6.73,6.07,6.38,6.2092,"3,944,300"
4,"May 2, 2018",6.30,6.75,6.17,6.74,6.5595,"4,852,500"
...,...,...,...,...,...,...,...
83,"Jan 8, 2018",4.40,4.47,4.24,4.46,4.3406,"2,576,100"
84,"Jan 5, 2018",4.08,4.39,4.03,4.36,4.2433,"3,298,400"
85,"Jan 4, 2018",4.18,4.29,3.91,4.24,4.1265,"3,950,500"
86,"Jan 3, 2018",3.83,4.30,3.82,4.16,4.0486,"5,485,800"


In [541]:
wti_oil = wti_oil_raw[["Date", "Close"]]
wti_oil

,Date,Close
0,"May 8, 2018",7.22
1,"May 7, 2018",6.96
2,"May 4, 2018",6.64
3,"May 3, 2018",6.38
4,"May 2, 2018",6.74
...,...,...
83,"Jan 8, 2018",4.46
84,"Jan 5, 2018",4.36
85,"Jan 4, 2018",4.24
86,"Jan 3, 2018",4.16


In [542]:
sales['Date']

0          2018-02-22
1          2018-01-11
2          2018-04-19
3          2018-04-06
4          2018-04-30
              ...    
6758120    2018-03-06
6758121    2018-03-02
6758122    2018-03-04
6758123    2018-03-12
6758124    2018-01-13
Name: Date, Length: 6758125, dtype: object

#### Convertir a Datetime

In [543]:
# Conversión a datetime
wti_oil["Date"] = pd.to_datetime(wti_oil["Date"], format='%b %d, %Y')
sales["Date"] = pd.to_datetime(sales['Date'])
wti_oil

C:\Users\andta\AppData\Local\Temp\ipykernel_5196\1901108212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_oil["Date"] = pd.to_datetime(wti_oil["Date"], format='%b %d, %Y')


,Date,Close
0,2018-05-08,7.22
1,2018-05-07,6.96
2,2018-05-04,6.64
3,2018-05-03,6.38
4,2018-05-02,6.74
...,...,...
83,2018-01-08,4.46
84,2018-01-05,4.36
85,2018-01-04,4.24
86,2018-01-03,4.16


In [544]:
# Join
sales = sales.merge(wti_oil, on="Date", how="outer")

sales["Close"].info()




<class 'pandas.core.series.Series'>
RangeIndex: 6758125 entries, 0 to 6758124
Series name: Close
Non-Null Count    Dtype  
--------------    -----  
4563612 non-null  float64
dtypes: float64(1)
memory usage: 51.6 MB


In [545]:
# Crear una nueva columna de mes (tipo Periodo mensual)
wti_oil["Month"] = wti_oil["Date"].dt.to_period("M")
# Por cada mes obtener la media del precio del wti
media_mensual = wti_oil.groupby("Month")["Close"].mean()


C:\Users\andta\AppData\Local\Temp\ipykernel_5196\627303586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_oil["Month"] = wti_oil["Date"].dt.to_period("M")


In [546]:
sales["Date"] = pd.to_datetime(sales["Date"])
# Crear una columna de mes (tipo Periodo mensual) en 'sales'
sales["Month"] = sales["Date"].dt.to_period("M")
# Rellenar valor nan por media del mes obtenido anteriormente

# Creo un diccionario
media_mensual_dict = media_mensual.to_dict()


# Rellenar los NaN de 'wti' con la media del mes correspondiente
sales["Close"] = sales["Close"].fillna(sales["Month"].map(media_mensual_dict))
sales["Close"].info()

# Encontramos valores Nan en las fechas, que venían del principio al extraer los datos de sales.csv
# Eliminarlos en el Análisis descriptivo
sales[sales["Close"].isna()]["Date"]

# Columna WTI renombrada
sales.rename(columns = {"Close" : "wti"}, inplace=True)



<class 'pandas.core.series.Series'>
RangeIndex: 6758125 entries, 0 to 6758124
Series name: Close
Non-Null Count    Dtype  
--------------    -----  
6690599 non-null  float64
dtypes: float64(1)
memory usage: 51.6 MB


In [547]:
sales.shape

(6758125, 30)

### Eliminar bloque tras explicar

In [548]:
# Extrae solo el mes
# wti_oil['Month'] = wti_oil['Date'].dt.month

# #Pasamos los mes a nombre
# wti_oil['Month_Name'] = wti_oil['Date'].dt.month_name()

# # Sustituimos Date por Month_name
# wti_oil['Date'] = wti_oil['Month_Name']

# if 'Month_Name' in wti_oil.columns:
#     wti_oil = wti_oil.drop(columns=['Month_Name'])

# if 'Month' in wti_oil.columns:
#     wti_oil = wti_oil.drop(columns=['Month'])

# wti_oil

In [549]:
# # Agrupar por 'Date' (meses) y calcular la media de la columna 'Close'
# wti_oil_final = wti_oil.groupby('Date', as_index=False)['Close'].mean()

# wti_oil_final['Date'] = wti_oil_final['Date'].str.lower()

# wti_oil_final.rename(columns={'Date': 'mes_nombre'}, inplace=True)

# wti_oil_final.head(12)

In [550]:
# # Realizamos el merge utilizando 'mes_nombre' de ambos DataFrames
# if 'Close' not in sales.columns:
#     sales = sales.merge(wti_oil_final[['mes_nombre', 'Close']], on='mes_nombre', how='left')

# # Ver el resultado
# sales.head()


# Tasa de desempleo + (((((((SALES)))))))

In [551]:
url_td = os.path.join(current_dir, "../data/raw/macrodata/tasa_desempleo_por_estados.csv")
td = pd.read_csv(url_td, sep=",")

td.head()

,Place,Year,Month,Unemployment Rate
0,Alabama,2018,May,3.5
1,Alabama,2018,April,3.5
2,Alabama,2018,March,4.0
3,Alabama,2018,February,4.4
4,Alabama,2018,January,4.4


In [552]:
# td["Month"] = pd.to_datetime(td["Month"])
# td["Month"]

In [553]:
# #Juntar la columna de year y month
# td['mes_ano'] = td.apply(lambda x: f"{x['Month']} {x['Year']}", axis=1)
# td

In [554]:
# #Cambiar nombre de columnas
# td.rename(columns = {"mes_ano" : "Date" }, inplace = True)
# td

In [555]:
# # Asegúrate de tener la columna como string (por si acaso)
# td['Date'] = td['Date'].astype(str)
# # Convertir a datetime (pandas infiere el formato si es consistente)
# td['Date'] = pd.to_datetime(td['Date'], errors='coerce')
# # Extraer nombre del mes
# td['mes_nombre'] = td['Date'].dt.month_name()
# td

In [556]:
# # Si el separador es coma, como "Houston, Texas"
# td['State'] = td['Place'].str.split(',').str[-1].str.strip()
# td
# #Eliminar columnas que ya no hacen falta
# td = td.drop(['Place', 'Year', 'Month'], axis=1)
# td

In [557]:
# import pandas as pd

# # 1. Verificación inicial de formatos
# print("Formato de fechas en 'sales':", sales['Date'].head(2))
# print("\nFormato de fechas en 'td':", td['Date'].head(2))

# # 2. Conversión segura a datetime
# def convert_to_datetime(df, date_col='Date'):
#     try:
#         # Primero intentamos el formato más común (MMM YYYY -> May 2018)
#         df[date_col] = pd.to_datetime(df[date_col], format='%b %Y', errors='coerce')
        
#         # Si hay valores nulos, probamos otro formato común (YYYY-MM-DD)
#         if df[date_col].isnull().any():
#             df[date_col] = pd.to_datetime(df[date_col], format='%Y-%m-%d', errors='coerce')
            
#         # Si aún hay nulos, intentamos parseo automático
#         if df[date_col].isnull().any():
#             df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
#     except Exception as e:
#         print(f"Error al convertir {date_col}: {str(e)}")
#     return df

# # Aplicamos a ambos DataFrames
# sales = convert_to_datetime(sales)
# td = convert_to_datetime(td)

# # 3. Verificación post-conversión
# print("\nTipos después de conversión:")
# print("sales['Date']:", sales['Date'].dtype)
# print("td['Date']:", td['Date'].dtype)

# print("\nValores nulos después de conversión:")
# print("sales:", sales['Date'].isnull().sum())
# print("td:", td['Date'].isnull().sum())


# # 5. Unión final
# sales_final = sales.merge(
#     td,
#     on=['State', 'Date'],
#     how='left',
#     suffixes=('_sales', '_td')
# )

# print("\nMerge completado con éxito!")
# print("Filas resultantes:", len(sales_final))

# population_density [OK]

In [558]:
import pandas as pd
url_pd = os.path.join(current_dir, "../data/raw/macrodata/population_density.csv")
population_density = pd.read_csv(url_pd, sep=",")

population_density.head()

,State,Population_2018
0,.Arizona,7158024.0
1,.Arkansas,3009733.0
2,.California,39461588.0
3,.Colorado,5691287.0
4,.Connecticut,3571520.0


In [559]:
population_density['State'] = population_density['State'].str.replace(".", "", regex=False)

population_density.head()

,State,Population_2018
0,Arizona,7158024.0
1,Arkansas,3009733.0
2,California,39461588.0
3,Colorado,5691287.0
4,Connecticut,3571520.0


In [560]:
sales = sales.merge(population_density,on="State",  how="left") 
sales

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State,wti,Month,Population_2018
0,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,19077,98726,25,15,0.1,0.0,NaT,DNPDCTGZSONAGKEWLEQ7,Thomas,R,Rangel,6,72 West White First Way,Austin,781,32,United States,AR,Texas,NaN,NaT,28628666.0
1,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,125395,77870,20,15,0.0,0.0,NaT,RSG6YQO7W8ZG9MUWJTLC,Oscar,B,Young,36,38 First Way,Detroit,72819,32,United States,AR,Michigan,NaN,NaT,9984072.0
2,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,136141,91603,24,15,0.0,0.0,NaT,J5U5RBSY3GOYU7LBMGH9,Mary,G,Herrera,50,670 White Oak Way,Cincinnati,83634,32,United States,AR,Ohio,NaN,NaT,11676341.0
3,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,139321,6206,2,15,0.0,0.0,NaT,0DKN419DSBL9Y2BWSDPL,Ebony,P,Hancock,59,45 White New Parkway,Kansas,56647,32,United States,AR,Kansas,NaN,NaT,2911359.0
4,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,188395,19057,5,15,0.0,0.0,NaT,RPPCBUJ4ZXY5JJA4HKK3,Lynn,K,Bonilla,61,39 Nobel Boulevard,Miami,6794,32,United States,AR,Florida,NaN,NaT,21244317.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758120,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6394853,75807,20,452,0.0,0.0,2018-05-09,79MKCW58DPTOI4YX00C0,Rene,M,Marquez,89,441 Second Parkway,St. Paul,77998,32,United States,AR,Minnesota,6.706667,2018-05,5606249.0
6758121,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6405665,79017,21,452,0.0,0.0,2018-05-09,DWDVNUBC0B25F050CL09,Fernando,U,Garrison,41,80 Milton Blvd.,Bakersfield,13915,32,United States,AR,California,6.706667,2018-05,39461588.0
6758122,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6466620,68070,18,452,0.0,0.0,2018-05-09,47VINHEN7JUTQ1BM63IJ,Rafael,V,Pugh,80,26 White Oak Drive,New Orleans,35640,32,United States,AR,Louisiana,6.706667,2018-05,4659690.0
6758123,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6710873,60319,16,452,0.0,0.0,2018-05-09,LMZ88LO2X6DICJNTF3GD,Emily,P,Garcia,73,38 Hague Blvd.,Santa Ana,23419,32,United States,AR,California,6.706667,2018-05,39461588.0


# personal_income

In [561]:
url_pincome = os.path.join(current_dir, "../data/raw/macrodata/personal_income.csv")
pincome = pd.read_csv(url_pincome, sep=",")

pincome.head()

,Estado,Ingresos_personales_2018
0,"Autauga, AL",2371211
1,"Baldwin, AL",10139295
2,"Barbour, AL",842907
3,"Bibb, AL",672223
4,"Blount, AL",2072951


In [562]:
# Paso 1: Crear un diccionario de abreviaciones a nombres completos
if 'State' not in pincome.columns:
    us_state_abreviatura = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}

    # Paso 2: Extraer el código del estado
    pincome['State_abreviatura'] = pincome['Estado'].str.split(',').str[1].str.strip()

    # Paso 3: Convertirlo a nombre completo
    pincome['State'] = pincome['State_abreviatura'].map(us_state_abreviatura)

    # Paso 4: Limpiar columnas
    pincome.drop(columns=['Estado', 'State_abreviatura'], inplace=True)

    # Paso 5: Renombrar
pincome.rename(columns={'Ingresos_personales_2018': 'personal_income'}, inplace=True)

pincome

,personal_income,State
0,2371211,Alabama
1,10139295,Alabama
2,842907,Alabama
3,672223,Alabama
4,2072951,Alabama
...,...,...
3135,2342826,Wyoming
3136,4966434,Wyoming
3137,812614,Wyoming
3138,371514,Wyoming


In [563]:
# Verificar si hay valores no numéricos en 'personal_income'
pincome['personal_income'] = pd.to_numeric(pincome['personal_income'], errors='coerce')

# Ahora 'personal_income' tiene NaN en los lugares donde había valores no numéricos

# Verificar nuevamente si hay algún valor no numérico
pincome_clean = pincome.dropna(subset=['personal_income'])

# Agrupar por 'State' y calcular la media de 'personal_income'
pincome_media_estados = pincome_clean.groupby('State')['personal_income'].mean().reset_index()

# Mostrar el resultado
print(pincome_media_estados)

             State  personal_income
0          Alabama     3.069764e+06
1           Alaska     4.224895e+06
2          Arizona     2.367116e+07
3         Arkansas     1.719611e+06
4       California     4.156992e+07
5         Colorado     3.842107e+06
6      Connecticut     3.257277e+07
7         Delaware     1.705012e+07
8          Florida     1.608971e+07
9          Georgia     3.088603e+06
10          Hawaii     2.247248e+07
11           Idaho     1.739308e+06
12        Illinois     7.085162e+06
13         Indiana     3.403557e+06
14            Iowa     1.562485e+06
15          Kansas     1.403880e+06
16        Kentucky     1.557759e+06
17       Louisiana     3.317512e+06
18           Maine     4.028949e+06
19        Maryland     1.532630e+07
20   Massachusetts     3.456817e+07
21        Michigan     5.713350e+06
22       Minnesota     3.650170e+06
23     Mississippi     1.364852e+06
24        Missouri     2.487690e+06
25         Montana     8.991432e+05
26        Nebraska     1.075

In [564]:
sales = sales.merge(pincome_media_estados[['State', 'personal_income']], on='State', how='left')

sales

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State,wti,Month,Population_2018,personal_income
0,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,19077,98726,25,15,0.1,0.0,NaT,DNPDCTGZSONAGKEWLEQ7,Thomas,R,Rangel,6,72 West White First Way,Austin,781,32,United States,AR,Texas,NaN,NaT,28628666.0,5.762952e+06
1,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,125395,77870,20,15,0.0,0.0,NaT,RSG6YQO7W8ZG9MUWJTLC,Oscar,B,Young,36,38 First Way,Detroit,72819,32,United States,AR,Michigan,NaN,NaT,9984072.0,5.713350e+06
2,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,136141,91603,24,15,0.0,0.0,NaT,J5U5RBSY3GOYU7LBMGH9,Mary,G,Herrera,50,670 White Oak Way,Cincinnati,83634,32,United States,AR,Ohio,NaN,NaT,11676341.0,6.392306e+06
3,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,139321,6206,2,15,0.0,0.0,NaT,0DKN419DSBL9Y2BWSDPL,Ebony,P,Hancock,59,45 White New Parkway,Kansas,56647,32,United States,AR,Kansas,NaN,NaT,2911359.0,1.403880e+06
4,1,Confections,"Spoon - Soup, Plastic",32.4420,Low,2017-03-03 09:47:09.310,Weak,True,0.0,188395,19057,5,15,0.0,0.0,NaT,RPPCBUJ4ZXY5JJA4HKK3,Lynn,K,Bonilla,61,39 Nobel Boulevard,Miami,6794,32,United States,AR,Florida,NaN,NaT,21244317.0,1.608971e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758120,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6394853,75807,20,452,0.0,0.0,2018-05-09,79MKCW58DPTOI4YX00C0,Rene,M,Marquez,89,441 Second Parkway,St. Paul,77998,32,United States,AR,Minnesota,6.706667,2018-05,5606249.0,3.650170e+06
6758121,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6405665,79017,21,452,0.0,0.0,2018-05-09,DWDVNUBC0B25F050CL09,Fernando,U,Garrison,41,80 Milton Blvd.,Bakersfield,13915,32,United States,AR,California,6.706667,2018-05,39461588.0,4.156992e+07
6758122,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6466620,68070,18,452,0.0,0.0,2018-05-09,47VINHEN7JUTQ1BM63IJ,Rafael,V,Pugh,80,26 White Oak Drive,New Orleans,35640,32,United States,AR,Louisiana,6.706667,2018-05,4659690.0,3.317512e+06
6758123,11,Produce,Napkin White - Starched,28.5653,Low,2017-01-06 14:39:42.890,Durable,Unknown,0.0,6710873,60319,16,452,0.0,0.0,2018-05-09,LMZ88LO2X6DICJNTF3GD,Emily,P,Garcia,73,38 Hague Blvd.,Santa Ana,23419,32,United States,AR,California,6.706667,2018-05,39461588.0,4.156992e+07


# population_growth [OK]

In [565]:
url_pgrowth = os.path.join(current_dir, "../data/raw/macrodata/population_growth.csv")
pgrowth = pd.read_csv(url_pgrowth, sep=",")

pgrowth.head()

,Estado,Poblacion_2017,Poblacion_2018,Crecimiento (%)
0,.Arizona,7044008.0,7158024.0,1.62
1,.Arkansas,3001345.0,3009733.0,0.28
2,.California,39358497.0,39461588.0,0.26
3,.Colorado,5611885.0,5691287.0,1.41
4,.Connecticut,3573297.0,3571520.0,-0.05


- Se eliminan los puntos que aparecían en los nombres de los estados

In [566]:
pgrowth['Estado'] = pgrowth['Estado'].str.replace(".", "", regex=False)

pgrowth.head()

,Estado,Poblacion_2017,Poblacion_2018,Crecimiento (%)
0,Arizona,7044008.0,7158024.0,1.62
1,Arkansas,3001345.0,3009733.0,0.28
2,California,39358497.0,39461588.0,0.26
3,Colorado,5611885.0,5691287.0,1.41
4,Connecticut,3573297.0,3571520.0,-0.05


- Renombramos la columna "Estado" por "State"

In [567]:
pgrowth.rename(columns = {"Estado" : "State" }, inplace = True)

- Dejamos en el DataFrame solo las 2 columnas que nos interesan

In [568]:
pgrowth_final = pgrowth[['State','Crecimiento (%)']]

pgrowth_final.head()

,State,Crecimiento (%)
0,Arizona,1.62
1,Arkansas,0.28
2,California,0.26
3,Colorado,1.41
4,Connecticut,-0.05


- Aplicamos merge por State

In [569]:
sales = sales.merge(pgrowth_final[['State', 'Crecimiento (%)']], on='State', how='left')

sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State,wti,Month,Population_2018,personal_income,Crecimiento (%)
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,19077,98726,25,15,0.1,0.0,NaT,DNPDCTGZSONAGKEWLEQ7,Thomas,R,Rangel,6,72 West White First Way,Austin,781,32,United States,AR,Texas,NaN,NaT,28628666.0,5.762952e+06,1.18
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,125395,77870,20,15,0.0,0.0,NaT,RSG6YQO7W8ZG9MUWJTLC,Oscar,B,Young,36,38 First Way,Detroit,72819,32,United States,AR,Michigan,NaN,NaT,9984072.0,5.713350e+06,0.11
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,136141,91603,24,15,0.0,0.0,NaT,J5U5RBSY3GOYU7LBMGH9,Mary,G,Herrera,50,670 White Oak Way,Cincinnati,83634,32,United States,AR,Ohio,NaN,NaT,11676341.0,6.392306e+06,0.14
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,139321,6206,2,15,0.0,0.0,NaT,0DKN419DSBL9Y2BWSDPL,Ebony,P,Hancock,59,45 White New Parkway,Kansas,56647,32,United States,AR,Kansas,NaN,NaT,2911359.0,1.403880e+06,0.09
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,188395,19057,5,15,0.0,0.0,NaT,RPPCBUJ4ZXY5JJA4HKK3,Lynn,K,Bonilla,61,39 Nobel Boulevard,Miami,6794,32,United States,AR,Florida,NaN,NaT,21244317.0,1.608971e+07,1.34
